In [1]:
%matplotlib qt

In [3]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# --- SETTINGS ---
CLEANED_CSVS = [
    "TDK_E_and_Toroid_Cores_no_catalog_cleaned.csv"        # add/remove files here
]

ID_COL = "Part No."
L_target = 3.5e-6
N_max = 10
I_amp = 100
PLOT_LIMIT = 999

# Y-tick step sizes (tweak these to densify/sparsify the axes)
YTICK_STEP_N  = 1        # step for Needed Turns (N)
YTICK_STEP_L  = 1        # step for Inductance (µH)
YTICK_STEP_V  = 10000    # step for Effective Volume (mm³)
YTICK_STEP_PV = 50       # step for Material Pv (kW/m³)

# Inductance calculation preferences
FORCE_MU_EFF = False     # True -> always use µ_eff path when possible, even if A_L exists
USE_AL_DEFAULT = True    # True -> if A_L exists, prefer it by default

# --- Material Pv overlay (100 kHz, 200 mT, 100 °C) ---
SHOW_MATERIAL_PV = True  # show right-axis line of Pv (kW/m³)

# kW/m^3 values from your table (duplicates collapsed; unknowns default to 0)
PV_KW_PER_M3_LOOKUP = {
    "N27": 920, "N41": 1400, "N51": 450, "N87": 375, "N88": 430, "N95": 350, "N96": 390,
    "N97": 300, "N49": 0, "PC200": 0, "N92": 410, "N91": 600, "N72": 540, "T38": 0,
    "T57": 0, "M33": 0, "M34": 0, "N48": 0, "K1": 0, "N22": 0, "N30": 0, "T35": 0,
    "T36": 0, "T37": 0, "T65": 0, "E11": 0, "E12": 0, "E16": 0, "E19": 0, "E13": 0, "E14": 0, "K10": 0
}

# ---------------
mu0 = 4e-7*np.pi

# Load & combine multiple cleaned CSVs
frames = []
for path in CLEANED_CSVS:
    if not os.path.exists(path):
        print(f"Warning: file not found → {path}")
        continue
    df_i = pd.read_csv(path)
    df_i["Source"] = os.path.basename(path)  # optional
    frames.append(df_i)

if not frames:
    raise SystemExit("No input CSVs found. Check CLEANED_CSVS list.")

df = pd.concat(frames, ignore_index=True)

# Ensure numeric (create as NaN if missing)
for col in ["Ae_m2","le_m","gap_m","A_L_H_per_turn2","mu_i","B_s_T","Ve_m3"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    else:
        df[col] = np.nan

# ---------- Detect core shape column ----------
SHAPE_CANDIDATES = [
    "Core Shape", "Core shape", "Core_Shape", "core_shape",
    "Shape", "Core Type", "Core type", "CoreType", "Family"
]
shape_col = next((c for c in SHAPE_CANDIDATES if c in df.columns), None)
if shape_col is None:
    shape_col = "Core_Shape"
    df[shape_col] = "Unknown"

# ---------- Detect material column & normalize codes ----------
MATERIAL_CANDIDATES = ["material_name", "Material Name", "Material", "Material_Name", "MaterialName"]
mat_col = next((c for c in MATERIAL_CANDIDATES if c in df.columns), None)

def _material_code(x: str) -> str | None:
    if pd.isna(x):
        return None
    s = str(x).upper().strip()
    # pull the first token that looks like a TDK material code (Nxx, Txx, Mxx, Exx, Kxx, PCxxx)
    m = re.search(r"\b(PC\d+|[NTMEK]\d{1,3})\b", s)
    if m:
        return m.group(1)
    # fallback: if the whole string matches a key
    if s in PV_KW_PER_M3_LOOKUP:
        return s
    return None

if mat_col is not None:
    df["Material_Code"] = df[mat_col].apply(_material_code)
else:
    df["Material_Code"] = None

# ---------- Validation & Row Skipping ----------
def _row_capabilities(row):
    def _pos(x):
        return (pd.notna(x)) and (x > 0)

    has_Ae   = _pos(row["Ae_m2"])
    has_Bs   = _pos(row["B_s_T"])
    has_AL   = _pos(row["A_L_H_per_turn2"])

    has_gap  = _pos(row["gap_m"])
    has_le   = _pos(row["le_m"])
    has_mu   = _pos(row["mu_i"])

    has_gap_geom = has_gap and has_le and has_mu and has_Ae
    can_L = has_gap_geom or has_AL
    can_B = has_Ae and has_Bs

    missing = []
    if not can_L:
        l_reqs = []
        if not has_AL:
            l_reqs.append("A_L_H_per_turn2")
        gap_needs = []
        if not has_gap: gap_needs.append("gap_m>0")
        if not has_le:  gap_needs.append("le_m")
        if not has_mu:  gap_needs.append("mu_i")
        if not has_Ae:  gap_needs.append("Ae_m2")
        if gap_needs:
            l_reqs.append(" / ".join(gap_needs))
        if l_reqs:
            missing.append("L-path: " + ", ".join(l_reqs))
    if not can_B:
        if not has_Ae: missing.append("Ae_m2")
        if not has_Bs: missing.append("B_s_T")

    return (can_L and can_B), missing

skip_msgs = []
def _mark_valid(row):
    ok, missing = _row_capabilities(row)
    if not ok:
        pn = row.get(ID_COL, "<unknown part>")
        msg = f"Skipping {pn} → missing: {', '.join(missing)}"
        skip_msgs.append(msg)
        return False
    return True

df["Row_Valid"] = df.apply(_mark_valid, axis=1)
if skip_msgs:
    print("\n".join(skip_msgs))
    print(f"\nSummary: skipped {len(skip_msgs)} row(s) due to missing required inputs.\n")

dfv = df[df["Row_Valid"]].copy()
if dfv.empty:
    raise SystemExit("All rows were invalid (missing required inputs). Nothing to compute.")

# ---------- Inductance & B-field ----------
def L_for_row_N(row, N):
    """
    Returns (L_H, mode) where mode is:
      'AL'              -> used datasheet A_L
      'gap'             -> used µ_eff from gap + geometry
      'gap_forced'      -> µ_eff used because FORCE_MU_EFF=True
      'none'            -> insufficient data
    """
    has_AL = pd.notna(row["A_L_H_per_turn2"]) and (row["A_L_H_per_turn2"] > 0)

    has_geom = (
        (row.get("gap_m", np.nan) is not np.nan) and (row["gap_m"] or 0) >= 0 and
        pd.notna(row.get("Ae_m2", np.nan)) and (row["Ae_m2"] or 0) > 0 and
        pd.notna(row.get("le_m", np.nan)) and (row["le_m"] or 0) > 0 and
        pd.notna(row.get("mu_i", np.nan)) and (row["mu_i"] or 0) > 0
    )

    # Helper for µ_eff computation (supports gap=0 too)
    def _L_from_mu_eff():
        mu_eff = 1.0 / ((1.0 / row["mu_i"]) + (row["gap_m"] / row["le_m"])) if (row["gap_m"] or 0) > 0 \
                 else row["mu_i"]
        L = mu0 * mu_eff * (N**2) * row["Ae_m2"] / row["le_m"]
        return L

    # 1) Forced µ_eff path (overrides A_L if available)
    if FORCE_MU_EFF and has_geom:
        return _L_from_mu_eff(), "gap_forced"

    # 2) Default: prefer A_L when available
    if USE_AL_DEFAULT and has_AL:
        return row["A_L_H_per_turn2"] * (N**2), "AL"

    # 3) Fallback: use µ_eff if geometry is available
    if has_geom:
        return _L_from_mu_eff(), "gap"

    # 4) Nothing usable
    return np.nan, "none"


def needed_turns_row(row):
    for N in range(1, N_max+1):
        L, mode = L_for_row_N(row, N)
        if np.isfinite(L) and L >= L_target:
            return pd.Series({"Needed_Turns": N, "L_H": L, "L_mode": mode})
    return pd.Series({"Needed_Turns": np.nan, "L_H": np.nan, "L_mode": "none"})

dfv[["Needed_Turns","L_H","L_mode"]] = dfv.apply(needed_turns_row, axis=1)
dfv["L_uH"] = dfv["L_H"] * 1e6
dfv["Meets_L"] = dfv["L_H"] >= L_target

valid_B = dfv["Needed_Turns"].notna() & dfv["Ae_m2"].gt(0)
dfv.loc[valid_B, "B_T"] = (dfv.loc[valid_B, "L_H"] * I_amp) / (dfv.loc[valid_B, "Needed_Turns"] * dfv.loc[valid_B, "Ae_m2"])
dfv["B_ratio"] = dfv["B_T"] / dfv["B_s_T"]
dfv["Meets_B"] = (dfv["B_T"] <= dfv["B_s_T"]) & dfv["B_T"].notna()

# ---------- Material Pv (kW/m^3) ----------
def _pv_kw_per_m3(code: str | None) -> float:
    if not code:
        return 0.0
    return float(PV_KW_PER_M3_LOOKUP.get(code, 0))

dfv["PV_kW_PER_M3"] = dfv["Material_Code"].apply(_pv_kw_per_m3)

# ---------- Winners ----------
winners = dfv[dfv["Meets_L"] & dfv["Meets_B"] & (dfv["Needed_Turns"] < N_max)].copy()
print(f"Winners: {len(winners)}")
if winners.empty:
    raise SystemExit("No cores meet all targets across the provided CSVs.")

# --- SAVE WINNERS CSV ---
if "Ve_m3" in winners.columns:
    winners["Ve_mm3"] = winners["Ve_m3"] * 1e9

cols = [
    "Source", ID_COL, "Catalog / Data Sheet",
    "Material_Code",
    "Needed_Turns", "L_H", "L_uH", "L_mode",
    "Ae_m2", "le_m", "gap_m", "mu_i", "A_L_H_per_turn2",
    "B_T", "B_s_T", "B_ratio",
    "Ve_m3", "Ve_mm3", "PV_kW_PER_M3", shape_col
]
cols = [c for c in cols if c in winners.columns]
winners_out = winners[cols].sort_values(["Needed_Turns", ID_COL])
winners_out.to_csv("winners_100A.csv", index=False)
print("Saved winners → winners_100A.csv")

# --- Colors by Core Shape ---
shapes = winners[shape_col].fillna("Unknown").astype(str)
unique_shapes = shapes.unique()
cmap = plt.cm.tab20.colors
shape_color_map = {s: cmap[i % len(cmap)] for i, s in enumerate(unique_shapes)}

# --- Prepare for plotting ---
to_plot = winners.sort_values(["Needed_Turns", ID_COL]).head(PLOT_LIMIT).copy()
x_pos = np.arange(len(to_plot))
to_plot["Ve_mm3"] = to_plot["Ve_m3"] * 1e9
to_plot["_shape"] = to_plot[shape_col].fillna("Unknown").astype(str)

# --- Figure ---
fig, (ax1, ax3) = plt.subplots(
    2, 1, figsize=(14, 10),
    gridspec_kw={'height_ratios': [2, 1]}, sharex=True
)

# --- Helpers for y-ticks ---
def _ticks(max_val, step):
    """0-based 'nice' ticks up to >= max_val."""
    if not np.isfinite(max_val) or max_val <= 0:
        max_val = step
    upper = np.ceil(max_val / step) * step
    return np.arange(0, upper + step/2, step)

def _ticks_from(start, max_val, step):
    """Start-based ticks (inclusive), up to >= max_val."""
    if not np.isfinite(step) or step <= 0:
        raise ValueError("step must be > 0")
    if not np.isfinite(start):
        start = 0.0
    if not np.isfinite(max_val):
        max_val = start + step
    top = max(start, max_val)
    n_steps = np.ceil((top - start) / step)
    upper = start + n_steps * step
    return np.arange(start, upper + step/2, step)

# --- Top subplot: Needed turns (color by shape, hatch for gap) ---
for i, (idx, row) in enumerate(to_plot.iterrows()):
    face = shape_color_map[row["_shape"]]
    if (row["gap_m"] or 0) > 0:
        ax1.bar(i, row["Needed_Turns"], color=face, hatch='///', edgecolor='black')
    else:
        ax1.bar(i, row["Needed_Turns"], color=face, edgecolor='black')

ax1.set_ylabel("Needed Turns (N)")
max_turns = np.nanmax(to_plot["Needed_Turns"].to_numpy(dtype=float))
ax1.set_yticks(_ticks(max_turns, YTICK_STEP_N))
ax1.grid(True, axis="y", linestyle="--", alpha=0.7)

# Inductance line (right axis of ax1) — start at target L
ax2 = ax1.twinx()
ax2.plot(x_pos, to_plot["L_uH"], color="red", marker="o", label="Inductance (µH)")
ax2.set_ylabel("Inductance (µH)")
L_target_uH = L_target * 1e6
max_L = np.nanmax(to_plot["L_uH"].to_numpy(dtype=float))
l_ticks = _ticks_from(L_target_uH, max_L, YTICK_STEP_L)
ax2.set_yticks(l_ticks)
ax2.set_ylim(L_target_uH, l_ticks[-1] if len(l_ticks) else L_target_uH + YTICK_STEP_L)

# Legends: shape colors + gap status
shape_handles = [Patch(facecolor=shape_color_map[s], edgecolor='black', label=s) for s in unique_shapes]
legend1 = ax1.legend(handles=shape_handles, title="Core shape", loc="upper left", ncols=1)
ax1.add_artist(legend1)

gap_handles = [
    Patch(facecolor='white', hatch='///', edgecolor='black', label="Gapped"),
    Patch(facecolor='white', edgecolor='black', label="Ungapped")
]
ax1.legend(handles=gap_handles, title="Gap", loc="upper right")

# --- Bottom subplot: Ve (mm³) colored by shape + material Pv overlay ---
for i, (idx, row) in enumerate(to_plot.iterrows()):
    ax3.bar(i, row["Ve_mm3"], color=shape_color_map[row["_shape"]], edgecolor='black')

ax3.set_ylabel("Effective volume Ve (mm³)")
ax3.grid(True, axis="y", linestyle="--", alpha=0.7)
max_V = np.nanmax(to_plot["Ve_mm3"].to_numpy(dtype=float))
ax3.set_yticks(_ticks(max_V, YTICK_STEP_V))

# Pv overlay (right y-axis), from lookup (kW/m³)
if SHOW_MATERIAL_PV and "PV_kW_PER_M3" in to_plot.columns and to_plot["PV_kW_PER_M3"].notna().any():
    ax4 = ax3.twinx()
    pv_line = ax4.plot(
        x_pos, to_plot["PV_kW_PER_M3"],
        color="black", marker="s", linewidth=1.5, label="Material Pv (kW/m³)"
    )
    ax4.set_ylabel("Material Pv (kW/m³) @ 100 kHz, 200 mT, 100 °C")
    max_PV = np.nanmax(to_plot["PV_kW_PER_M3"].to_numpy(dtype=float))
    ax4.set_yticks(_ticks(max_PV, YTICK_STEP_PV))
    ax3.legend(handles=[Patch(facecolor=shape_color_map[s], edgecolor='black', label=s) for s in unique_shapes] + [pv_line[0]],
               loc="upper right")
else:
    ax3.legend(handles=[Patch(facecolor=shape_color_map[s], edgecolor='black', label=s) for s in unique_shapes],
               loc="upper right")

# Shared x labels
ax3.set_xticks(x_pos)
ax3.set_xticklabels(to_plot[ID_COL], rotation=45, ha="right")

# Title (notes the Pv spec)
pv_note = " | Pv lookup @ 100 kHz, 200 mT, 100 °C" if SHOW_MATERIAL_PV else ""
plt.suptitle(
    f"Cores meeting ALL targets: L≥{L_target*1e6:.1f} µH, N<{N_max}, B≤B_s (I={I_amp:.1f}A){pv_note}"
)
plt.tight_layout()
plt.show()


Skipping B64290A0711X830 (N30) → missing: Ae_m2
Skipping B64290L0022X830 (N30) → missing: Ae_m2
Skipping B64290L0038X830 (N30) → missing: Ae_m2
Skipping B64290L0040X830 (N30) → missing: Ae_m2
Skipping B64290L0042X830 (N30) → missing: Ae_m2
Skipping B64290L0043X830 (N30) → missing: Ae_m2
Skipping B64290L0044X830 (N30) → missing: Ae_m2
Skipping B64290L0045X830 (N30) → missing: Ae_m2
Skipping B64290L0048X830 (N30) → missing: Ae_m2
Skipping B64290L0058X830 (N30) → missing: Ae_m2
Skipping B64290L0062X830 (N30) → missing: Ae_m2
Skipping B64290L0082X830 (N30) → missing: Ae_m2
Skipping B64290L0084X830 (N30) → missing: Ae_m2
Skipping B64290L0615X830 (N30) → missing: Ae_m2
Skipping B64290L0616X830 (N30) → missing: Ae_m2
Skipping B64290L0618X830 (N30) → missing: Ae_m2
Skipping B64290L0623X830 (N30) → missing: Ae_m2
Skipping B64290L0626X830 (N30) → missing: Ae_m2
Skipping B64290L0631X830 (N30) → missing: Ae_m2
Skipping B64290L0638X830 (N30) → missing: Ae_m2
Skipping B64290L0644X830 (N30) → missing